In [ ]:
import numpy as np
import sympy as sp
from General import functies as functies
from General import classes as classes
from matplotlib import pyplot as plt
from scipy.interpolate import CubicSpline

In [ ]:
#de exacte oplossing van die fucking derdegraadsveelterm die [H+] beschrijft
H, Ka, Na, c0, a = sp.symbols('H Ka Na c0 a') #a = concentratie H en OH door autoprotolyse van water
Kw = 10**(-7) #de waterconstante
""" #poging die fout bleek
vergelijking = sp.Eq(H**3 + Ka*H**2 + (Na-Kw*c0)*H + Ka*(Na - Kw), 0)
display(vergelijking)
oplossing = sp.solve(vergelijking, H)
for opl in oplossing:
    testval = opl.subs(Ka, 10**(-4.74)).subs(Na, 0).subs(c0, 1).evalf()
    display(testval)
    print('pH:')
    display(-np.log10(float(abs(testval))))
    print('_________')
    #skip dat imaginair deel, uit de 
print('____________take 2_______________')
#poging 2:"""
vergelijking = sp.Eq(Ka*(c0 + a)*(a+Na)**2 - Kw**2 + Kw*a*(a+Na), 0)
oplossing = sp.solve(vergelijking, a)
#testreeks 1 van de oplossingen:
""""
for opl in oplossing:
    H = Kw/(opl+Na) - opl
    testval = H.subs(Ka, 10**(-4.74)).subs(Na, 0).subs(c0,1).evalf()
    display(testval)
    print('pH:')
    display(-np.log10(float(abs(testval))))
    print('______')
#gezien hier pH = 1/2 pKa - 1/2 log cA = 4.74/2, valt de eerste oplossing al af
for opl in oplossing:
    H = Kw/(opl+Na) - opl
    testval = H.subs(Ka, 10**(-4.74)).subs(Na, 1/2).subs(c0,1).evalf()
    display(testval)
    print('pH:')
    display(-np.log10(float(abs(testval))))
    print('______')
#gezien hier pH = pKa - log(cA/cB) = 4.74 moet het de derde oplossing zijn die we zoeken"""

exacteoplossing = sp.simplify(oplossing[2].subs(Ka, 10**(-4.74)))
display(exacteoplossing)

In [ ]:
def pH_buffer(pKa, ca, cb):
    return pKa - np.log10(ca/cb)

def nieuw_mengsel(ca, cb, deltaca, deltacb):
    canieuw = ca+deltaca-deltacb
    cbnieuw = cb-deltaca+deltacb
    return canieuw, cbnieuw

def pH_experiment_bufferzone(VNaOH, parameters):
    begin_n = parameters[0]
    NaOH = VNaOH * 0.1 #Molair
    return pH_buffer(4.74, (begin_n - NaOH)/(40e-3 + VNaOH), NaOH/(40e-3 + VNaOH))

def pH_experiment(VNaOH, parameters):
    c0val = parameters[0]
    concentratie_CH3COOH = c0val * 40e-3/(40e-3 + VNaOH)
    nNaOH = VNaOH*0.1 #stofhoeveelheid natriumhydroxide
    cNaOH = nNaOH/(40e-3+VNaOH)
    output = sp.lambdify([c0, Na], exacteoplossing, 'numpy')(concentratie_CH3COOH, cNaOH)
    return output


    

In [ ]:
#voorbereidende oefening
cabegin = 0.5 #alles in molair doen
cbbegin = 0.5
pKa = 4.76
extrazuur = np.linspace(-0.5,0.5) #negatief toegevoegd zuur = toegevoegd base
fig, ax = plt.subplots(1,1)
ax.plot(extrazuur, pH_buffer(pKa, *nieuw_mengsel(cabegin, cbbegin, extrazuur, 0)), label = 'pH')
D1 = pH_buffer(pKa, *nieuw_mengsel(cabegin, cbbegin, extrazuur[:-1], 0))
D2 = pH_buffer(pKa, *nieuw_mengsel(cabegin, cbbegin, extrazuur[1:], 0))
afgeleide = (D2-D1)/(extrazuur[1]-extrazuur[0])
ax.plot(extrazuur[1:], afgeleide, label = 'afgeleide van de pH naar de concentratie zuur')
ax.set_xlabel('extra toegevoegd zuur (positief) of base (negatief) [M]')
ax.set_ylabel('pH/mol')
ax.legend()
plt.tight_layout()
plt.show()
print('minimum van de afgeleide:')
print('x = ',(extrazuur[np.argmax(afgeleide)] + extrazuur[1+np.argmax(afgeleide)])/2, ' dy/dx = ', np.max(afgeleide))
#het is duidelijk minimaal rond 0

In [ ]:
#de plot van de data en zo
adres = 'buffer/titratie2.txt'
volumes,pHs = np.loadtxt(adres, delimiter=",").T #volumes in ml
volumes *= 10**(-3) #vanaf nu volumes in l
volumelinspace = np.linspace(0,20e-3,5000)
pHfunctie = CubicSpline(volumes, pHs)
fig, ax = plt.subplots(1,1, figsize = (10,10))
ax.errorbar(volumes, pHs, xerr = 0.5e-3, yerr = 0.01, linestyle = '', label = 'data')
ax.plot(volumelinspace, pHfunctie(volumelinspace), label = 'interpolatie')
ax.plot(volumelinspace, pH_experiment(volumelinspace, [0.001]), label = 'model', zorder = 2)
ax.set_ylabel('pH')
ax.set_xlabel('Volume toegevoegd NaOH [l]')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
#fit overal:
def invals(x,y):
    return [0.001/40]
functies.fit_2D(['c0'], pH_experiment, invals, volumes, pHs, np.full(np.shape(volumes),1e-3)**2/12, np.full(np.shape(pHs),0.01)**2/12,  x_as_titel= 'toegevoegd volume (l)', y_as_titel = 'pH', bounds=[(0,5)], detailed_logs=False)

In [ ]:
#fit het buffergebied binnen pKa - 1 tot pKa + 1
indxmin = 0
while pHs[indxmin] < 3.74:
    indxmin+=1
indxmax = indxmin+1
while pHs[indxmax] < 5.74:
    indxmax += 1
volumesreduced = volumes[indxmin: indxmax]
pHsreduced = pHs[indxmin: indxmax]
def invals(x,y):
    return [0.001]
functies.fit_2D(['n_0'], pH_experiment, invals, volumesreduced, pHsreduced, np.full(np.shape(volumesreduced),1e-3)**2/12, np.full(np.shape(pHsreduced),0.01)**2/12, grootteorde = 1e-3, x_as_titel= 'toegevoegd volume (l)', y_as_titel = 'pH', bounds=[(0,5)], detailed_logs=False)